In [5]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("cbs.db")

# csv_file = "ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025.csv"  # Update this path
# chunk_size = 500000  # You can adjust this based on memory

# for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
#     chunk.to_sql("ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025", conn, if_exists="append", index=False)

In [6]:
# Preview the first 5 rows
query = "SELECT BRANCH_CODE, BRANCH_NAME FROM STTM_BRANCH"
df = pd.read_sql_query(query, conn)
df.head(3)

,BRANCH_CODE,BRANCH_NAME
0,16,TUNDUMA BRANCH
1,17,RWAGASORE AGENCY
2,19,MOROGORO BRANCH


In [7]:
# SQL query to fetch the specified columns with PRODUCT starting with 'DW'
query = """
SELECT 
    CAST(AC_BRANCH AS TEXT) AS AC_BRANCH,
    AC_CCY,
    AC_NO,
    FCY_AMOUNT,
    LCY_AMOUNT,
    AMOUNT_TAG,
    PRODUCT,
    RELATED_ACCOUNT,
    TRN_CODE,
    TRN_DT,
    TRN_REF_NO,
    VALUE_DT
FROM ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025
WHERE 
    PRODUCT LIKE 'DW%' AND 
    CUST_GL = 'A' AND 
    AMOUNT_TAG = 'AMT_EQUIV' AND
    AC_BRANCH != 999;
"""

# Execute query and load into DataFrame
df = pd.read_sql_query(query, conn)

df.head()

,AC_BRANCH,AC_CCY,AC_NO,FCY_AMOUNT,LCY_AMOUNT,AMOUNT_TAG,PRODUCT,RELATED_ACCOUNT,TRN_CODE,TRN_DT,TRN_REF_NO,VALUE_DT
0,5,TZS,005000175399,0.0,832000.0,AMT_EQUIV,DWTB,None,W05,05/02/2025 00:00:00,005DWTB251220001,05/01/2025 00:00:00
1,1,TZS,001000373006,0.0,45000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,001DWTW251220501,05/01/2025 00:00:00
2,1,TZS,001000373006,0.0,50000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,001DWTW251220502,05/01/2025 00:00:00
3,32,TZS,032000035874,0.0,10000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,032DWTW251220002,05/01/2025 00:00:00
4,1,TZS,001000373006,0.0,100000.0,AMT_EQUIV,DWTB,None,W05,05/02/2025 00:00:00,001DWTB251220502,05/01/2025 00:00:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AC_BRANCH        573 non-null    object 
 1   AC_CCY           573 non-null    object 
 2   AC_NO            573 non-null    object 
 3   FCY_AMOUNT       573 non-null    float64
 4   LCY_AMOUNT       573 non-null    float64
 5   AMOUNT_TAG       573 non-null    object 
 6   PRODUCT          573 non-null    object 
 7   RELATED_ACCOUNT  0 non-null      object 
 8   TRN_CODE         573 non-null    object 
 9   TRN_DT           573 non-null    object 
 10  TRN_REF_NO       573 non-null    object 
 11  VALUE_DT         573 non-null    object 
dtypes: float64(2), object(10)
memory usage: 53.8+ KB


In [5]:
# # Ensure TRN_DT is datetime
# df["TRN_DT"] = pd.to_datetime(df["TRN_DT"])

# # Step 1: Group to get total LCY_AMOUNT per account per day
# grouped = df.groupby(["TRN_DT", "AC_NO"]).agg({
#     "LCY_AMOUNT": "sum",
#     "AC_BRANCH": "first",
#     "PRODUCT": "first",
#     "RELATED_CUSTOMER": "first",
#     "VALUE_DT": "first"
# }).reset_index()

# # Step 2: Get the account with the highest total per day
# top_per_day = grouped.loc[grouped.groupby("TRN_DT")["LCY_AMOUNT"].idxmax()].reset_index(drop=True)

# # Step 3: Sort and keep required columns
# top_per_day = top_per_day.sort_values("TRN_DT")[[
#     "AC_BRANCH", "AC_NO", "LCY_AMOUNT", "PRODUCT", "RELATED_CUSTOMER", "TRN_DT", "VALUE_DT"
# ]]

# # Format AC_BRANCH to 3-digit strings (e.g., '001')
# top_per_day['AC_BRANCH'] = top_per_day['AC_BRANCH'].apply(
#     lambda x: str(int(x)).zfill(3) if pd.notnull(x) else ''
# )

# # Format RELATED_CUSTOMER to 9-digit strings (e.g., '000123456')
# top_per_day['RELATED_CUSTOMER'] = top_per_day['RELATED_CUSTOMER'].apply(
#     lambda x: str(int(x)).zfill(9) if pd.notnull(x) else ''
# )

# # Step 4: Export to CSV
# top_per_day.to_csv("top_account_per_day.csv", index=False)

# # Show preview
# top_per_day.head(500)

In [9]:
df.shape

(573, 12)

In [10]:
df.head()

,AC_BRANCH,AC_CCY,AC_NO,FCY_AMOUNT,LCY_AMOUNT,AMOUNT_TAG,PRODUCT,RELATED_ACCOUNT,TRN_CODE,TRN_DT,TRN_REF_NO,VALUE_DT
0,5,TZS,005000175399,0.0,832000.0,AMT_EQUIV,DWTB,None,W05,05/02/2025 00:00:00,005DWTB251220001,05/01/2025 00:00:00
1,1,TZS,001000373006,0.0,45000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,001DWTW251220501,05/01/2025 00:00:00
2,1,TZS,001000373006,0.0,50000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,001DWTW251220502,05/01/2025 00:00:00
3,32,TZS,032000035874,0.0,10000.0,AMT_EQUIV,DWTW,None,W06,05/02/2025 00:00:00,032DWTW251220002,05/01/2025 00:00:00
4,1,TZS,001000373006,0.0,100000.0,AMT_EQUIV,DWTB,None,W05,05/02/2025 00:00:00,001DWTB251220502,05/01/2025 00:00:00
